In [1]:
import cobra
from cobra.io import load_model
from pathlib import Path
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
import logging
import pandas as pd
from cobra import Model, Reaction, Metabolite
from cobra.io import write_sbml_model, validate_sbml_model

In [2]:
%cd C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\MODELS\OTHER_SPECIES\Phaseolus vulgaris
pvulModel=read_sbml_model('pvulModel_CheckError.xml')

C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\MODELS\OTHER_SPECIES\Phaseolus vulgaris


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


In [3]:
submodel=Model()
submodel.add_reaction(pvulModel.reactions.get_by_id('RXN-16463'))

# SE CREA UN SUBMODELO DE PRUEBA

In [4]:
for i in submodel.metabolites:
    #submodel.add_boundary(submodel.metabolites.get_by_id(i.id), type="demand")
    #submodel.add_boundary(submodel.metabolites.get_by_id(i.id), type="sink")


IndentationError: expected an indented block (598450908.py, line 3)

In [5]:
submodel.reactions

[<Reaction RXN-16463 at 0x1f7a483c070>]

In [6]:
submodel.metabolites

[<Metabolite ACP[c] at 0x1f7a3dc4610>,
 <Metabolite ATP[c] at 0x1f7a3dc4220>,
 <Metabolite CPD-17686[c] at 0x1f7a3ef49d0>,
 <Metabolite _5-hexenoyl-ACPs[c] at 0x1f7a3d9ef10>,
 <Metabolite AMP[c] at 0x1f7a3dc4e80>,
 <Metabolite PPI[c] at 0x1f7a41ac490>]

## SE REVISA QUE ESTE BLOQUEADA LA RXN


In [7]:
blockedrxns=cobra.flux_analysis.find_blocked_reactions(submodel)
blockedrxns

['RXN-16463']

## SE AGREGAN METABOLITOS EXTRACELULARES

In [8]:
#def compartment_metabolite(modelo):
mets=list(submodel.metabolites)
for i in mets:
    met=submodel.metabolites.get_by_id(i.id)
    dup=met.copy()
    dup.id=dup.id.replace('[c]',"[e]")
    dup.compartment='e'
    submodel.add_metabolites([dup])
        

In [9]:
submodel.metabolites

[<Metabolite ACP[c] at 0x1f7a3dc4610>,
 <Metabolite ATP[c] at 0x1f7a3dc4220>,
 <Metabolite CPD-17686[c] at 0x1f7a3ef49d0>,
 <Metabolite _5-hexenoyl-ACPs[c] at 0x1f7a3d9ef10>,
 <Metabolite AMP[c] at 0x1f7a3dc4e80>,
 <Metabolite PPI[c] at 0x1f7a41ac490>,
 <Metabolite ACP[e] at 0x1f7a4865190>,
 <Metabolite ATP[e] at 0x1f7a4856b20>,
 <Metabolite CPD-17686[e] at 0x1f7a48652b0>,
 <Metabolite _5-hexenoyl-ACPs[e] at 0x1f78a6c7ca0>,
 <Metabolite AMP[e] at 0x1f7a485eb80>,
 <Metabolite PPI[e] at 0x1f7a485e490>]

## SE AGREGAN RXN EXCHANGE PARA LOS e METS

In [10]:
for i in submodel.metabolites:
    if i.compartment == 'e':
        submodel.add_boundary(i, type="exchange")

In [11]:
blockedrxns=cobra.flux_analysis.find_blocked_reactions(submodel)
blockedrxns

['RXN-16463',
 'EX_ACP[e]',
 'EX_ATP[e]',
 'EX_CPD-17686[e]',
 'EX__5-hexenoyl-ACPs[e]',
 'EX_AMP[e]',
 'EX_PPI[e]']

In [56]:
#r=list(submodel.reactions.get_by_id('EX_ACP[e]').metabolites)
#r

[<Metabolite ACP[e] at 0x2bf69078640>]

## SE CREAN LAS RXNS DE TRANSPORTE

In [12]:
def create_transportrxns(rxnEx):
    transport_rxn = Reaction("Trans_"+rxnEx.id)
    transport_rxn.name = "Transporte"+rxnEx.name
    met=list(rxnEx.metabolites)
    transport_rxn.add_metabolites({
        submodel.metabolites.get_by_id(met[0].id): -1,  # glucose (extracellular) on the left side of the reaction
        submodel.metabolites.get_by_id(met[0].id[:-3]+"[c]"): 1    # glucose (cytoplasmic) on the right side of the reaction
    })

    # Set the lower and upper bounds of the transport reaction
    transport_rxn.lower_bound = -1000
    transport_rxn.upper_bound = 1000

    # Add the transport reaction to the model
    reactionstoAdd.append(transport_rxn)

In [13]:
reactionstoAdd=[]
for i in submodel.reactions:
    if i.id[-3:]=='[e]':
        print(i)
        reaction=create_transportrxns(i)
        
        
        
        

EX_ACP[e]: ACP[e] <=> 
EX_ATP[e]: ATP[e] <=> 
EX_CPD-17686[e]: CPD-17686[e] <=> 
EX__5-hexenoyl-ACPs[e]: _5-hexenoyl-ACPs[e] <=> 
EX_AMP[e]: AMP[e] <=> 
EX_PPI[e]: PPI[e] <=> 


## SE AGREGAN LA RXNS DE TRANSPORTE AL SUBMODEL


In [14]:
submodel.add_reactions(reactionstoAdd)

In [15]:
trans=submodel.reactions.get_by_id('Trans_EX_ACP[e]')
trans

Reaction identifier,Trans_EX_ACP[e]
Name,Transportea soluble [acyl-carrier protein] exchange
Memory address,0x1f7a48c0d90
Stoichiometry,ACP[e] <=> ACP[c] a soluble [acyl-carrier protein] <=> a soluble [acyl-carrier protein]
GPR,
Lower bound,-1000
Upper bound,1000


In [16]:
for i in submodel.reactions:
    print(i)

RXN-16463: ACP[c] + ATP[c] + CPD-17686[c] <=> AMP[c] + PPI[c] + _5-hexenoyl-ACPs[c]
EX_ACP[e]: ACP[e] <=> 
EX_ATP[e]: ATP[e] <=> 
EX_CPD-17686[e]: CPD-17686[e] <=> 
EX__5-hexenoyl-ACPs[e]: _5-hexenoyl-ACPs[e] <=> 
EX_AMP[e]: AMP[e] <=> 
EX_PPI[e]: PPI[e] <=> 
Trans_EX_ACP[e]: ACP[e] <=> ACP[c]
Trans_EX_ATP[e]: ATP[e] <=> ATP[c]
Trans_EX_CPD-17686[e]: CPD-17686[e] <=> CPD-17686[c]
Trans_EX__5-hexenoyl-ACPs[e]: _5-hexenoyl-ACPs[e] <=> _5-hexenoyl-ACPs[c]
Trans_EX_AMP[e]: AMP[e] <=> AMP[c]
Trans_EX_PPI[e]: PPI[e] <=> PPI[c]


In [17]:
submodel.objective="RXN-16463"
sol=submodel.optimize().objective_value
submodel.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ACP[e],EX_ACP[e],1000,0,0.00%
ATP[e],EX_ATP[e],1000,10,62.50%
CPD-17686[e],EX_CPD-17686[e],1000,6,37.50%
Metabolite,Reaction,Flux,C-Number,C-Flux
AMP[e],EX_AMP[e],-1000,10,62.50%
PPI[e],EX_PPI[e],-1000,0,0.00%
_5-hexenoyl-ACPs[e],EX__5-hexenoyl-ACPs[e],-1000,6,37.50%


<bound method Model.summary of <Model None at 0x2bf6a865fa0>>
